In [ ]:
!git clone https://github.com/Shenggan/BCCD_Dataset.git
%cd BCCD_Dataset


Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 7.44 MiB/s, done.
Resolving deltas: 100% (378/378), done.
/content/BCCD_Dataset


In [ ]:
import os
import xml.etree.ElementTree as ET

# Paths
annotations_dir = '/content/BCCD_Dataset/BCCD/Annotations'
labels_dir = '/content/BCCD_Dataset/labels'
os.makedirs(labels_dir, exist_ok=True)

# Classes in BCCD Dataset
classes = ["RBC", "WBC", "Platelets"]

# Function to convert VOC to YOLO
def convert_voc_to_yolo(annotations_dir, labels_dir):
    for file in os.listdir(annotations_dir):
        if not file.endswith('.xml'):
            continue

        tree = ET.parse(os.path.join(annotations_dir, file))
        root = tree.getroot()

        image_width = int(root.find("size/width").text)
        image_height = int(root.find("size/height").text)

        label_file_path = os.path.join(labels_dir, file.replace('.xml', '.txt'))
        with open(label_file_path, "w") as label_file:
            for obj in root.findall("object"):
                class_name = obj.find("name").text
                if class_name not in classes:
                    continue

                class_id = classes.index(class_name)
                bbox = obj.find("bndbox")
                xmin = int(bbox.find("xmin").text)
                ymin = int(bbox.find("ymin").text)
                xmax = int(bbox.find("xmax").text)
                ymax = int(bbox.find("ymax").text)

                # Convert to YOLO format
                x_center = (xmin + xmax) / 2 / image_width
                y_center = (ymin + ymax) / 2 / image_height
                width = (xmax - xmin) / image_width
                height = (ymax - ymin) / image_height

                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

convert_voc_to_yolo(annotations_dir, labels_dir)
print("Annotations converted to YOLO format.")


Annotations converted to YOLO format.


In [ ]:
from albumentations import Compose, RandomRotate90, Flip, ShiftScaleRotate, Blur, CLAHE
from albumentations.pytorch import ToTensorV2
from PIL import Image
import cv2

# Augmentation pipeline
def augment_image(image_path, save_path):
    image = cv2.imread(image_path)
    augmentations = Compose([
        RandomRotate90(),
        Flip(),
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.7),
        Blur(blur_limit=3, p=0.1),
        CLAHE(p=0.1),
    ])
    augmented = augmentations(image=image)
    cv2.imwrite(save_path, augmented["image"])

# Example usage
augment_image('/content/BCCD_Dataset/BCCD/JPEGImages/BloodImage_00000.jpg', '/content/augmented_image.jpg')


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
<ipython-input-3-171d6b0af572>:11: DeprecationWarning: Flip is deprecated. Consider using HorizontalFlip, VerticalFlip, RandomRotate90 or D4.
  Flip(),


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 17075 (delta 29), reused 30 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17075/17075), 15.69 MiB | 18.09 MiB/s, done.
Resolving deltas: 100% (11724/11724), done.
/content/BCCD_Dataset/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 20.3 MB/s eta 0:00:00


In [ ]:
%cd /content/BCCD_Dataset

/content/BCCD_Dataset


In [ ]:
!mkdir -p /content/BCCD_Dataset/train/images /content/BCCD_Dataset/train/labels
!mkdir -p /content/BCCD_Dataset/val/images /content/BCCD_Dataset/val/labels


In [ ]:
import os
import shutil

# Paths
base_dir = '/content/BCCD_Dataset/BCCD'
images_dir = os.path.join(base_dir, 'JPEGImages')
labels_dir = '/content/BCCD_Dataset/labels'
splits_dir = os.path.join(base_dir, 'ImageSets', 'Main')

train_images_dir = '/content/BCCD_Dataset/train/images'
train_labels_dir = '/content/BCCD_Dataset/train/labels'
val_images_dir = '/content/BCCD_Dataset/val/images'
val_labels_dir = '/content/BCCD_Dataset/val/labels'

# Create the directories
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Function to move files based on split
def move_files(split_file, images_output_dir, labels_output_dir):
    with open(split_file, 'r') as f:
        for line in f:
            base_name = line.strip()
            image_file = os.path.join(images_dir, f"{base_name}.jpg")
            label_file = os.path.join(labels_dir, f"{base_name}.txt")

            if os.path.exists(image_file):
                shutil.move(image_file, os.path.join(images_output_dir, f"{base_name}.jpg"))
            if os.path.exists(label_file):
                shutil.move(label_file, os.path.join(labels_output_dir, f"{base_name}.txt"))

# Move train files
move_files(os.path.join(splits_dir, 'train.txt'), train_images_dir, train_labels_dir)

# Move val files
move_files(os.path.join(splits_dir, 'val.txt'), val_images_dir, val_labels_dir)

print("Files moved successfully!")


Files moved successfully!


In [ ]:
%cd /content/BCCD_Dataset/yolov5/

/content/BCCD_Dataset/yolov5


In [ ]:
!python train.py \
    --img 640 \
    --batch 16 \
    --epochs 20 \
    --data data/bccd.yaml \
    --weights yolov5s.pt \
    --project BCCD_Training \
    --name yolov5_bccd


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-11 05:58:57.386811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 05:58:57.407117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 05:58:57.413363: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [ ]:
import torch
from PIL import Image
import numpy as np
import cv2

# Load the fine-tuned model
def load_model(weights_path="BCCD_Training/yolov5_bccd/weights/best.pt"):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)
    return model

# Perform inference and get labeled image
def predict_image(image_path, model):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')

    # Perform inference
    results = model(image)

    # Get predictions
    # Move the tensor to CPU before converting to NumPy
    predictions = results.xyxy[0].cpu().numpy()  # Format: [x1, y1, x2, y2, confidence, class]

    # Render the image with bounding boxes and labels
    rendered_image = results.render()[0]

    # Convert rendered image from numpy array to PIL image for better visualization
    labeled_image = Image.fromarray(rendered_image)

    return predictions, labeled_image

# Display the labeled image (optional step, can be used in Gradio/Streamlit)
def show_image(image):
    image.show()

# Example usage
if __name__ == "__main__":
    model = load_model()  # Load the fine-tuned model
    image_path = '/content/augmented_image.jpg'  # Path to the input image
    predictions, labeled_image = predict_image(image_path, model)
    print("Predictions:", predictions)  # Print the bounding box predictions and classes
    show_image(labeled_image)  # Show the image with labels and bounding boxes

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-12-11 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Predictions: [[     142.99      66.059      230.01      170.33      0.8042           0]
 [     245.12      146.65      488.58       390.5     0.78154           1]
 [     61.102       317.5      159.29      431.53     0.69932           0]
 [     196.97      348.71      299.67      440.36     0.66466           0]
 [     514.58      23.735      626.91      130.38      0.6585           0]
 [     538.51      333.12         640      450.26     0.65604           0]
 [     365.61           0      479.57      106.07     0.65406           0]
 [      392.2      342.76      498.84      441.06     0.64905           0]
 [     450.17      395.45      551.98         480     0.64307           0]
 [          0      76.705      90.077      163.34     0.56939           0]
 [          0      328.35      77.127      434.09     0.54063           0]
 [      471.3      114.64      583.47      228.31     0.50392           0]
 [     51.499      217.64      150.14      330.19     0.49272           0]
 [          

In [ ]:
import torch
from PIL import Image
import numpy as np
import cv2

# Load the fine-tuned model
def load_model(weights_path="BCCD_Training/yolov5_bccd/weights/best.pt"):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path, force_reload=True)
    return model

# Perform inference and get labeled image
def predict_image(image_path, model):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')

    # Perform inference
    results = model(image)

    # Get predictions
    predictions = results.xyxy[0].cpu().numpy()  # Format: [x1, y1, x2, y2, confidence, class]

    # Render the image with bounding boxes and labels
    rendered_image = results.render()[0]

    # Convert rendered image from numpy array to PIL image for better visualization
    labeled_image = Image.fromarray(rendered_image)

    return predictions, labeled_image

# Display the labeled image (optional step, can be used in Gradio/Streamlit)
def show_image(image):
    image.show()

# Example usage
if __name__ == "__main__":
    model = load_model()  # Load the fine-tuned model
    image_path = '/content/augmented_image.jpg'  # Path to the input image
    predictions, labeled_image = predict_image(image_path, model)
    print("Predictions:", predictions)  # Print the bounding box predictions and classes
    show_image(labeled_image)  # Show the image with labels and bounding boxes


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-12-11 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Predictions: [[     142.99      66.059      230.01      170.33      0.8042           0]
 [     245.12      146.65      488.58       390.5     0.78154           1]
 [     61.102       317.5      159.29      431.53     0.69932           0]
 [     196.97      348.71      299.67      440.36     0.66466           0]
 [     514.58      23.735      626.91      130.38      0.6585           0]
 [     538.51      333.12         640      450.26     0.65604           0]
 [     365.61           0      479.57      106.07     0.65406           0]
 [      392.2      342.76      498.84      441.06     0.64905           0]
 [     450.17      395.45      551.98         480     0.64307           0]
 [          0      76.705      90.077      163.34     0.56939           0]
 [          0      328.35      77.127      434.09     0.54063           0]
 [      471.3      114.64      583.47      228.31     0.50392           0]
 [     51.499      217.64      150.14      330.19     0.49272           0]
 [          

In [ ]:
!zip -r my_project.zip /content/


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2024.12.09/ (stored 0%)
  adding: content/.config/logs/2024.12.09/14.20.16.487772.log (deflated 58%)
  adding: content/.config/logs/2024.12.09/14.19.42.277947.log (deflated 93%)
  adding: content/.config/logs/2024.12.09/14.20.15.174962.log (deflated 87%)
  adding: content/.config/

In [ ]:
from google.colab import files
files.download('my_project.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>